<div style="display: flex; align-items: center; justify-content: center; flex-wrap: wrap;">
    <div style="flex: 1; max-width: 400px; display: flex; justify-content: center;">
        <img src="https://i.ibb.co/JBPWVYR/Logo-Nova-IMS-Black.png" style="max-width: 50%; height: auto; margin-top: 50px; margin-bottom: 50px;margin-left: 6rem;">
    </div>
    <div style="flex: 2; text-align: center; margin-top: 20px;margin-left: 8rem;">
        <div style="font-size: 28px; font-weight: bold; line-height: 1.2;">
            <span style="color: #22c1c3;">DL Project |</span> <span style="color: #08529C;">Predicting Rare Species from Images using Deep Learning</span>
        </div>
        <div style="font-size: 17px; font-weight: bold; margin-top: 10px;">
            Spring Semester | 2024 - 2025
        </div>
        <div style="font-size: 17px; font-weight: bold;">
            Master in Data Science and Advanced Analytics
        </div>
        <div style="margin-top: 20px;">
            <div>André Silvestre, 20240502</div>
            <div>Diogo Duarte, 20240525</div>
            <div>Filipa Pereira, 20240509</div>
            <div>Maria Cruz, 20230760</div>
            <div>Umeima Mahomed, 20240543</div>
        </div>
        <div style="margin-top: 20px; font-weight: bold;">
            Group 37
        </div>
    </div>
</div>

<div style="background: linear-gradient(to right, #22c1c3, #27b1dd, #2d9cfd, #090979); 
            padding: 1px; color: white; border-radius: 500px; text-align: center;">
</div>

## **📚 Libraries Import**

In [ ]:
# System imports
import os
import sys
import time
import datetime
from tqdm import tqdm
from typing_extensions import Self, Any      # For Python 3.10
# from typing import Self, Any               # For Python >3.11

from pathlib import Path

# Data manipulation imports
import numpy as np
import pandas as pd  
import warnings
warnings.filterwarnings("ignore")

# Data visualization imports
import matplotlib.pyplot as plt
import seaborn as sns

# Deep learning imports
import tensorflow as tf
from keras.ops import add
from keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import Model, Sequential, Input
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling, Lambda, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras import regularizers                                                                           # For L2 regularization
# import visualkeras

# Evaluation imports
from keras.metrics import CategoricalAccuracy, AUC, F1Score, Precision, Recall

# Other imports
from itertools import product

# Set the style of the visualization
pd.set_option('future.no_silent_downcasting', True)   # use int instead of float in DataFrame
pd.set_option("display.max_columns", None)            # display all columns

# Disable warnings (FutureWarning)
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# Set random seed for reproducibility
np.random.seed(2025)

In [ ]:
# Creates a SSL context that does not verify the server’s certificate - Needed for downloading pretrained models
# Source: https://precli.readthedocs.io/0.3.4/rules/python/stdlib/ssl_create_unverified_context.html
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
print("TensorFlow Version:", tf.__version__)
print("Is TensorFlow built with CUDA?", tf.test.is_built_with_cuda())
print("GPU Available:", tf.config.list_physical_devices('GPU'))
print("GPU Device Name:", tf.test.gpu_device_name())                                # (if error in Google Colab: Make sure your Hardware accelerator is set to GPU. 
                                                                                    # Runtime > Change runtime type > Hardware Accelerator)

In [ ]:
# Get build information from TensorFlow
build_info = tf.sysconfig.get_build_info()

print("TensorFlow version:", tf.__version__)
print("Python version:", sys.version)
print("CUDA version:", build_info.get("cuda_version", "Not available"))
print("cuDNN version:", build_info.get("cudnn_version", "Not available"))

In [ ]:
# Import custom module for importing data, visualization, and utilities
import utilities

## **🧮 Import Databases**

In [ ]:
# Define the path to the data
train_dir = Path("data/RareSpecies_Split/train")
val_dir = Path("data/RareSpecies_Split/val")
test_dir = Path("data/RareSpecies_Split/test")

# For Google Collab
# train_dir = Path("/content/RareSpecies_Split/train")
# val_dir = Path("/content/RareSpecies_Split/val")
# test_dir = Path("/content/RareSpecies_Split/test")

In [ ]:
# Image Generators 
n_classes = 202                                     # Number of classes (we already know this based on previous notebook)
image_size = (224, 224)                             # Image size (224x224)
img_height, img_width = image_size                  # Image dimensions
batch_size = 64                                     # Batch size
input_shape = (img_height, img_width, 3)            # Input shape of the model
value_range = (0.0, 1.0)                            # Range of pixel values

In [ ]:
# Get class names from directory
class_names = sorted(os.listdir(train_dir))
class_indices = {name: i for i, name in enumerate(class_names)}

# Import the image dataset from the directory
from utilities import load_images_from_directory
train_datagen, val_datagen, test_datagen = load_images_from_directory(train_dir, val_dir, test_dir,
                                                                      labels='inferred', label_mode='categorical',
                                                                      class_names=class_names, color_mode='rgb',
                                                                      batch_size=batch_size, image_size=image_size, seed=2025, 
                                                                      interpolation='bilinear', crop_to_aspect_ratio=False, pad_to_aspect_ratio=False)

print(f"\nLoaded: Train ({train_datagen.cardinality().numpy() * batch_size}), "
        f"Val ({val_datagen.cardinality().numpy() * batch_size}), "
        f"Test ({test_datagen.cardinality().numpy() * batch_size})")

In [ ]:
# Check the shape of the data (batch_size, img_width, img_height, 3)
for x, y in train_datagen.take(1):
    print("Train batch shape:", x.shape, y.shape)
for x, y in val_datagen.take(1):
    print("Val batch shape:", x.shape, y.shape)
for x, y in test_datagen.take(1):
    print("Test batch shape:", x.shape, y.shape)

# <a class='anchor' id='3'></a>
<br>
<style>
@import url('https://fonts.cdnfonts.com/css/avenir-next-lt-pro?styles=29974');
</style>

<div style="background: linear-gradient(to right, #22c1c3, #27b1dd, #2d9cfd, #090979); 
            padding: 10px; color: white; border-radius: 300px; text-align: center;">
    <center><h1 style="margin-left: 140px;margin-top: 10px; margin-bottom: 4px; color: white;
                       font-size: 32px; font-family: 'Avenir Next LT Pro', sans-serif;">
        <b>3 | Modeling - EfficientNetV2B0</b></h1></center>
</div>

<br><br>

<center><img src="https://www.researchgate.net/publication/356981443/figure/fig3/AS:1100238135791616@1639328621231/Architecture-of-EfficientNet-B0-with-MBConv-as-Basic-building-blocks.png" style="width: 700px"></center>

# **💡 Modeling**

In [ ]:
# Create directories for saving model checkpoints and evaluation logs
os.makedirs("./ModelCallbacks/7_EfficientNetV2B0", exist_ok=True)      # exist_ok=True | Create directory if it doesn't exist
os.makedirs("./ModelsEvaluation/7_EfficientNetV2B0", exist_ok=True)

In [ ]:
from tensorflow.keras.applications import EfficientNetV2B0

class RareSpeciesCNN_EfficientNetV2B0(Model):
    """Custom CNN for rare species classification using EfficientNetV2B0.
    
    Architecture: EfficientNetV2B0 
    """
    def __init__(self, n_classes=202,
                 apply_grayscale=False,
                 apply_contrast=False, contrast_factor=1.5,
                 apply_saturation=False, saturation_factor=1.5):
        super().__init__()                          # Call the parent class constructor

        # Store preprocessing flags and factors
        self.apply_grayscale = apply_grayscale
        self.apply_contrast = apply_contrast
        self.apply_saturation = apply_saturation

        # Preprocessing Layers (Same as in the previous notebook)
        self.rescale_layer = Rescaling(scale=1 / 255.0, name="Rescale_Layer")
        if self.apply_contrast:
            self.contrast_layer = Lambda(lambda x: tf.image.adjust_contrast(x, contrast_factor=contrast_factor), name='Adjust_Contrast') 
        if self.apply_saturation:
            self.saturation_layer = Lambda(lambda x: tf.image.adjust_saturation(x, saturation_factor=saturation_factor), name='Adjust_Saturation')
        if self.apply_grayscale:
            self.grayscale_layer = Lambda(lambda x: tf.image.rgb_to_grayscale(x), name='RGB_to_Grayscale')

        # Load EfficientNetV2B0 Pretrained Model
        self.efficientnetv2b0 = EfficientNetV2B0(include_top=False,      # Do not include the top classification layer (we will add our own because we have a different number of classes)
                                                 classes=n_classes,      # Number of classes
                                                 weights='imagenet')     # Use ImageNet weights
        self.efficientnetv2b0.trainable = False                          # Freeze the convolutional layers in the base model (transfer learning)
        
        # --- Classification Head ---
        self.global_avg_pool = GlobalAveragePooling2D(name="Global_Average_Pooling")      # Global Average Pooling layer
        self.dense1 = Dense(128, name="Dense_Layer1")                                     # Smaller intermediate dense layer
        self.dropout = Dropout(0.5, name="Dropout_Layer")                                 # Dropout layer for regularization
        self.dense_output = Dense(n_classes, activation='softmax', name="Output_Layer")   # Output layer with softmax activation (for multi-class classification)

    def call(self, inputs, training=False):
        # Apply mandatory rescaling
        x = self.rescale_layer(inputs)
        
        # Apply conditional preprocessing layers
        if self.apply_contrast:
            x = self.contrast_layer(x)
        if self.apply_saturation:
            x = self.saturation_layer(x)
        if self.apply_grayscale:
            x = self.grayscale_layer(x)

        # Pass through EfficientNetV2B0 model
        x = self.efficientnetv2b0(x, training=training)      # Pass through the model (EfficientNetV2B0)

        # Classification Head
        x = self.global_avg_pool(x)                    # Global Average Pooling
        x = self.dense1(x)                             # Dense layer
        x = self.dropout(x, training=training)         # Dropout layer
        outputs = self.dense_output(x)                 # Output layer
        return outputs

# Example Instantiation and Summary
model = RareSpeciesCNN_EfficientNetV2B0(
    n_classes=n_classes,
    apply_grayscale=False,
    apply_contrast=False,
    apply_saturation=False,
)

# Build the model by providing an input shape
inputs = Input(shape=(img_width, img_height, 3))        # Input shape
_ = model.call(inputs)                                  # Call the model to build it
model.summary()                                         # Print the model summary

---

# **🥇 Best Combinations Models**

#### **Original | Grayscale=F | Contrast=F | Saturation=F**

In [ ]:
# Retrain model with máx 100 epochs and EarlyStopping
# EfficientNetV2B0 Model - Original Dataset | Grayscale=F | Contrast=F | Saturation=F
model = RareSpeciesCNN_EfficientNetV2B0(
    n_classes=n_classes, 
    apply_grayscale=False, 
    apply_contrast=False,                         
    apply_saturation=False
)
# Build the model by providing an input shape
inputs = Input(shape=(img_width, img_height, 3))        # Input shape
_ = model.call(inputs)                                  # Call the model to build it
model.summary()                                         # Print the model summary

In [ ]:
# Compile model
optimizer = Adam(learning_rate=0.001, weight_decay=0.01, name="Optimizer")                                        # Adam for faster convergence
loss = CategoricalCrossentropy(name="Loss")                                                                       # Suitable for multi-class one-hot labels
metrics = [CategoricalAccuracy(name="accuracy"), Precision(name="precision"), Recall(name="recall"), F1Score(average="macro", name="f1_score"), AUC(name="auc")]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# Callbacks
model_name = f"RareSpeciesCNN_EfficientNetV2B0_{datetime.datetime.now().strftime('%Y%m%d')}_Original_maxEpochs100" # Model name
callbacks = [ModelCheckpoint(f"./ModelCallbacks/7_EfficientNetV2B0/checkpoint_{model_name}.keras", monitor="val_loss", save_best_only=True, verbose=0), CSVLogger(f"./ModelCallbacks/7_EfficientNetV2B0/metrics_{model_name}.csv"), LearningRateScheduler(lambda epoch, lr: lr * 0.95), EarlyStopping(monitor='val_loss', patience=3, verbose=1)]

In [ ]:
# Train model
start_time = time.time()
history = model.fit(train_datagen, batch_size = batch_size, epochs=100, validation_data=val_datagen, callbacks=callbacks, verbose=1)
train_time = round(time.time() - start_time, 2)
print(f"\nTraining completed in \033[1m{train_time} seconds ({str(datetime.timedelta(seconds=train_time))} h)\033[0m).")

##### **🧪 Model Selection & 📏 Model Evaluation**

In [ ]:
# Evaluate model
from utilities import plot_metrics
plot_metrics(history, file_path=f"./ModelsEvaluation/7_EfficientNetV2B0/7_TrainingValidationMetrics_{model_name}.png")

In [ ]:
# Evaluate on validation and test sets
train_results = {'accuracy': history.history['accuracy'][-1], 'precision': history.history['precision'][-1], 'recall': history.history['recall'][-1], 'f1_score': history.history['f1_score'][-1], 'auc': history.history['auc'][-1]}
val_results = model.evaluate(val_datagen, batch_size=batch_size, return_dict=True, verbose=1)
test_results = model.evaluate(test_datagen, batch_size=batch_size, return_dict=True, verbose=1)

In [ ]:
# Display results
from utilities import display_side_by_side, create_evaluation_dataframe
results_df = create_evaluation_dataframe(
    model_name="EfficientNetV2B0", variation="Original | Grayscale=F | Contrast=F | Saturation=F | Adam=0.001",   # Dataset | Grayscale | Contrast | Saturation | Optimizer=Learning Rate
    train_metrics=train_results, val_metrics=val_results, test_metrics=test_results, train_time=train_time,
    csv_save_path= f"./ModelsEvaluation/7_EfficientNetV2B0/7_TrainingValidationMetrics_{model_name}.csv"
)
display_side_by_side(results_df, super_title="Model Evaluation Results")

In [ ]:
# Plot n right and n wrong predictions
from utilities import plot_predictions
plot_predictions(
    model=model,
    class_names=class_names,
    train_dir=train_dir,
    test_data=test_datagen,
    num_images=10,
    file_path=f"./ModelsEvaluation/7_EfficientNetV2B0/7_TestPredictions_{model_name}.png",
)

----

#### **Original | Grayscale=F | Contrast=T | Saturation=F**

In [ ]:
# Retrain model with máx 100 epochs and EarlyStopping
# EfficientNetV2B0 Model - Original | Grayscale=F | Contrast=T | Saturation=F | Adam=0.001
model = RareSpeciesCNN_EfficientNetV2B0(
    n_classes=n_classes, 
    apply_grayscale=False, 
    apply_contrast=True,                         
    apply_saturation=False
)
# Build the model by providing an input shape
inputs = Input(shape=(img_width, img_height, 3))        # Input shape
_ = model.call(inputs)                                  # Call the model to build it
model.summary()                                         # Print the model summary

In [ ]:
# Compile model
optimizer = Adam(learning_rate=0.001, weight_decay=0.01, name="Optimizer")                                        # Adam for faster convergence
loss = CategoricalCrossentropy(name="Loss")                                                                       # Suitable for multi-class one-hot labels
metrics = [CategoricalAccuracy(name="accuracy"), Precision(name="precision"), Recall(name="recall"), F1Score(average="macro", name="f1_score"), AUC(name="auc")]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# Callbacks
model_name = f"RareSpeciesCNN_EfficientNetV2B0_{datetime.datetime.now().strftime('%Y%m%d')}_OriginalContrast_maxEpochs100" # Model name
callbacks = [ModelCheckpoint(f"./ModelCallbacks/7_EfficientNetV2B0/checkpoint_{model_name}.keras", monitor="val_loss", save_best_only=True, verbose=0), CSVLogger(f"./ModelCallbacks/7_EfficientNetV2B0/metrics_{model_name}.csv"), LearningRateScheduler(lambda epoch, lr: lr * 0.95), EarlyStopping(monitor='val_loss', patience=3, verbose=1)]

In [ ]:
# Train model
start_time = time.time()
history = model.fit(train_datagen, batch_size = batch_size, epochs=100, validation_data=val_datagen, callbacks=callbacks, verbose=1)
train_time = round(time.time() - start_time, 2)
print(f"\nTraining completed in \033[1m{train_time} seconds ({str(datetime.timedelta(seconds=train_time))} h)\033[0m).")

##### **🧪 Model Selection & 📏 Model Evaluation**

In [ ]:
# Evaluate model
from utilities import plot_metrics
plot_metrics(history, file_path=f"./ModelsEvaluation/7_EfficientNetV2B0/7_TrainingValidationMetrics_{model_name}.png")

In [ ]:
# Evaluate on validation and test sets
train_results = {'accuracy': history.history['accuracy'][-1], 'precision': history.history['precision'][-1], 'recall': history.history['recall'][-1], 'f1_score': history.history['f1_score'][-1], 'auc': history.history['auc'][-1]}
val_results = model.evaluate(val_datagen, batch_size=batch_size, return_dict=True, verbose=1)
test_results = model.evaluate(test_datagen, batch_size=batch_size, return_dict=True, verbose=1)

In [ ]:
# Display results
from utilities import display_side_by_side, create_evaluation_dataframe
results_df = create_evaluation_dataframe(
    model_name="EfficientNetV2B0", variation="Original | Grayscale=F | Contrast=T | Saturation=F | Adam=0.001",   # Dataset | Grayscale | Contrast | Saturation | Optimizer=Learning Rate
    train_metrics=train_results, val_metrics=val_results, test_metrics=test_results, train_time=train_time,
    csv_save_path= f"./ModelsEvaluation/7_EfficientNetV2B0/7_TrainingValidationMetrics_{model_name}.csv"
)
display_side_by_side(results_df, super_title="Model Evaluation Results")

In [ ]:
# Plot n right and n wrong predictions
from utilities import plot_predictions
plot_predictions(
    model=model,
    class_names=class_names,
    train_dir=train_dir,
    test_data=test_datagen,
    num_images=10,
    file_path=f"./ModelsEvaluation/7_EfficientNetV2B0/7_TestPredictions_{model_name}.png",
)

---

### **🖌️ SMOTE (Data Augmentation)**


In [ ]:
# Import SMOTE training data
train_DataAugmentationSMOTE_dir = Path("data/RareSpecies_Split/train_DataAugmentationSMOTE")
val_dir = Path("data/RareSpecies_Split/val")
test_dir = Path("data/RareSpecies_Split/test")

# train_DataAugmentationSMOTE_dir = Path("/content/RareSpecies_Split/train_DataAugmentationSMOTE")
# val_dir = Path("/content/RareSpecies_Split/val")
# test_dir = Path("/content/RareSpecies_Split/test")

# Import the image dataset from the directory
train_DataAugmentationSMOTE_datagen, val_datagen, test_datagen = load_images_from_directory(train_DataAugmentationSMOTE_dir, val_dir, test_dir,
                                                                      labels='inferred', label_mode='categorical',
                                                                      class_names=class_names, color_mode='rgb',
                                                                      batch_size=batch_size, image_size=image_size, seed=2025, 
                                                                      interpolation='bilinear', crop_to_aspect_ratio=False, pad_to_aspect_ratio=False)
# Check the shape of the data (batch_size, img_width, img_height, 3)
for x, y in train_DataAugmentationSMOTE_datagen.take(1):
    print("Train batch shape:", x.shape, y.shape)
for x, y in val_datagen.take(1):
    print("Val batch shape:", x.shape, y.shape)
for x, y in test_datagen.take(1):
    print("Test batch shape:", x.shape, y.shape)

#### **SMOTE | Grayscale=F | Contrast=F | Saturation=F**

In [ ]:
# EfficientNetV2B0 Model - SMOTE Data Augmentation | Grayscale=F | Contrast=F | Saturation=F
model = RareSpeciesCNN_EfficientNetV2B0(
    n_classes=n_classes, 
    apply_grayscale=False, 
    apply_contrast=False,                         
    apply_saturation=False
)

# Build the model by providing an input shape
inputs = Input(shape=(img_width, img_height, 3))        # Input shape
_ = model.call(inputs)                                  # Call the model to build it
model.summary()                                         # Print the model summary

In [ ]:
# Compile model
optimizer = Adam(learning_rate=0.001, weight_decay=0.01, name="Optimizer")
loss = CategoricalCrossentropy(name="Loss")
metrics = [CategoricalAccuracy(name="accuracy"), Precision(name="precision"), Recall(name="recall"), F1Score(average="macro", name="f1_score"), AUC(name="auc")]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# Callbacks
model_name = f"RareSpeciesCNN_EfficientNetV2B0_{datetime.datetime.now().strftime('%Y%m%d')}_SMOTE_MaxEpochs100" # Model name
callbacks = [ModelCheckpoint(f"./ModelCallbacks/7_EfficientNetV2B0/checkpoint_{model_name}.keras", monitor="val_loss", save_best_only=True, verbose=0), CSVLogger(f"./ModelCallbacks/7_EfficientNetV2B0/metrics_{model_name}.csv"), LearningRateScheduler(lambda epoch, lr: lr * 0.95), EarlyStopping(monitor='val_loss', patience=3, verbose=1)]

In [ ]:
# Train model
start_time = time.time()
history = model.fit(train_DataAugmentationSMOTE_datagen, batch_size=batch_size, epochs=100, validation_data=val_datagen, callbacks=callbacks, verbose=1)
train_time = round(time.time() - start_time, 2)
print(f"\nTraining completed in \033[1m{train_time} seconds ({str(datetime.timedelta(seconds=train_time))} h)\033[0m).")

##### **🧪 Model Selection & 📏 Model Evaluation**

In [ ]:
# Evaluate model
from utilities import plot_metrics
plot_metrics(history, file_path=f"./ModelsEvaluation/7_EfficientNetV2B0/7_TrainingValidationMetrics_{model_name}.png")

In [ ]:
# Evaluate on validation and test sets
train_results = {'accuracy': history.history['accuracy'][-1], 'precision': history.history['precision'][-1], 'recall': history.history['recall'][-1], 'f1_score': history.history['f1_score'][-1], 'auc': history.history['auc'][-1]}
val_results = model.evaluate(val_datagen, batch_size=batch_size, return_dict=True, verbose=1)
test_results = model.evaluate(test_datagen, batch_size=batch_size, return_dict=True, verbose=1)

In [ ]:
# Display results
from utilities import display_side_by_side, create_evaluation_dataframe
results_df = create_evaluation_dataframe(
    model_name="EfficientNetV2B0", variation="SMOTE | Grayscale=F | Contrast=F | Saturation=F | Adam=0.001",
    train_metrics=train_results, val_metrics=val_results, test_metrics=test_results, train_time=train_time,
    csv_save_path= f"./ModelsEvaluation/7_EfficientNetV2B0/7_TrainingValidationMetrics_{model_name}.csv"
)
display_side_by_side(results_df, super_title="Model Evaluation Results")

In [ ]:
# Plot n right and n wrong predictions
from utilities import plot_predictions
plot_predictions(
    model=model,
    class_names=class_names,
    train_dir=train_dir,
    test_data=test_datagen,
    num_images=10,
    file_path=f"./ModelsEvaluation/7_EfficientNetV2B0/7_TestPredictions_{model_name}.png",
)

---

#### **SMOTE | Grayscale=F | Contrast=T | Saturation=F**

In [ ]:
# EfficientNetV2B0 Model - SMOTE Data Augmentation | Grayscale=F | Contrast=T | Saturation=F
model = RareSpeciesCNN_EfficientNetV2B0(
    n_classes=n_classes, 
    apply_grayscale=False, 
    apply_contrast=True,                         
    apply_saturation=False
)
# Build the model by providing an input shape
inputs = Input(shape=(img_width, img_height, 3))        # Input shape
_ = model.call(inputs)                                  # Call the model to build it
model.summary()                                         # Print the model summary

In [ ]:
# Compile model
optimizer = Adam(learning_rate=0.001, weight_decay=0.01, name="Optimizer")
loss = CategoricalCrossentropy(name="Loss")
metrics = [CategoricalAccuracy(name="accuracy"), Precision(name="precision"), Recall(name="recall"), F1Score(average="macro", name="f1_score"), AUC(name="auc")]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# Callbacks
model_name = f"RareSpeciesCNN_EfficientNetV2B0_{datetime.datetime.now().strftime('%Y%m%d')}_SMOTEContrast_MaxEpochs100" # Model name
callbacks = [ModelCheckpoint(f"./ModelCallbacks/7_EfficientNetV2B0/checkpoint_{model_name}.keras", monitor="val_loss", save_best_only=True, verbose=0), CSVLogger(f"./ModelCallbacks/7_EfficientNetV2B0/metrics_{model_name}.csv"), LearningRateScheduler(lambda epoch, lr: lr * 0.95), EarlyStopping(monitor='val_loss', patience=3, verbose=1)]

In [ ]:
# Train model
start_time = time.time()
history = model.fit(train_DataAugmentationSMOTE_datagen, batch_size=batch_size, epochs=100, validation_data=val_datagen, callbacks=callbacks, verbose=1)
train_time = round(time.time() - start_time, 2)
print(f"\nTraining completed in \033[1m{train_time} seconds ({str(datetime.timedelta(seconds=train_time))} h)\033[0m).")

##### **🧪 Model Selection & 📏 Model Evaluation**

In [ ]:
# Evaluate model
from utilities import plot_metrics
plot_metrics(history, file_path=f"./ModelsEvaluation/7_EfficientNetV2B0/7_TrainingValidationMetrics_{model_name}.png")

In [ ]:
# Evaluate on validation and test sets
train_results = {'accuracy': history.history['accuracy'][-1], 'precision': history.history['precision'][-1], 'recall': history.history['recall'][-1], 'f1_score': history.history['f1_score'][-1], 'auc': history.history['auc'][-1]}
val_results = model.evaluate(val_datagen, batch_size=batch_size, return_dict=True, verbose=1)
test_results = model.evaluate(test_datagen, batch_size=batch_size, return_dict=True, verbose=1)

In [ ]:
# Display results
from utilities import display_side_by_side, create_evaluation_dataframe
results_df = create_evaluation_dataframe(
    model_name="EfficientNetV2B0", variation="SMOTE | Grayscale=F | Contrast=T | Saturation=F | Adam=0.001",
    train_metrics=train_results, val_metrics=val_results, test_metrics=test_results, train_time=train_time,
    csv_save_path= f"./ModelsEvaluation/7_EfficientNetV2B0/7_TrainingValidationMetrics_{model_name}.csv"
)
display_side_by_side(results_df, super_title="Model Evaluation Results")

In [ ]:
# Plot n right and n wrong predictions
from utilities import plot_predictions
plot_predictions(
    model=model,
    class_names=class_names,
    train_dir=train_dir,
    test_data=test_datagen,
    num_images=10,
    file_path=f"./ModelsEvaluation/7_EfficientNetV2B0/7_TestPredictions_{model_name}.png",
)

---

#### **SMOTE | Grayscale=F | Contrast=F | Saturation=T**

In [ ]:
# EfficientNetV2B0 Model - SMOTE Data Augmentation | Grayscale=F | Contrast=F | Saturation=T
model = RareSpeciesCNN_EfficientNetV2B0(
    n_classes=n_classes, 
    apply_grayscale=False, 
    apply_contrast=False,                         
    apply_saturation=True
)
# Build the model by providing an input shape
inputs = Input(shape=(img_width, img_height, 3))        # Input shape
_ = model.call(inputs)                                  # Call the model to build it
model.summary()                                         # Print the model summary

In [ ]:
# Compile model
optimizer = Adam(learning_rate=0.001, weight_decay=0.01, name="Optimizer")
loss = CategoricalCrossentropy(name="Loss")
metrics = [CategoricalAccuracy(name="accuracy"), Precision(name="precision"), Recall(name="recall"), F1Score(average="macro", name="f1_score"), AUC(name="auc")]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# Callbacks
model_name = f"RareSpeciesCNN_EfficientNetV2B0_{datetime.datetime.now().strftime('%Y%m%d')}_SMOTESaturation_MaxEpochs100" # Model name
callbacks = [ModelCheckpoint(f"./ModelCallbacks/7_EfficientNetV2B0/checkpoint_{model_name}.keras", monitor="val_loss", save_best_only=True, verbose=0), CSVLogger(f"./ModelCallbacks/7_EfficientNetV2B0/metrics_{model_name}.csv"), LearningRateScheduler(lambda epoch, lr: lr * 0.95), EarlyStopping(monitor='val_loss', patience=3, verbose=1)]

In [ ]:
# Train model
start_time = time.time()
history = model.fit(train_DataAugmentationSMOTE_datagen, batch_size=batch_size, epochs=100, validation_data=val_datagen, callbacks=callbacks, verbose=1)
train_time = round(time.time() - start_time, 2)
print(f"\nTraining completed in \033[1m{train_time} seconds ({str(datetime.timedelta(seconds=train_time))} h)\033[0m).")

##### **🧪 Model Selection & 📏 Model Evaluation**

In [ ]:
# Evaluate model
from utilities import plot_metrics
plot_metrics(history, file_path=f"./ModelsEvaluation/7_EfficientNetV2B0/7_TrainingValidationMetrics_{model_name}.png")

In [ ]:
# Evaluate on validation and test sets
train_results = {'accuracy': history.history['accuracy'][-1], 'precision': history.history['precision'][-1], 'recall': history.history['recall'][-1], 'f1_score': history.history['f1_score'][-1], 'auc': history.history['auc'][-1]}
val_results = model.evaluate(val_datagen, batch_size=batch_size, return_dict=True, verbose=1)
test_results = model.evaluate(test_datagen, batch_size=batch_size, return_dict=True, verbose=1)

In [ ]:
# Display results
from utilities import display_side_by_side, create_evaluation_dataframe
results_df = create_evaluation_dataframe(
    model_name="EfficientNetV2B0", variation="SMOTE | Grayscale=F | Contrast=F | Saturation=T | Adam=0.001",
    train_metrics=train_results, val_metrics=val_results, test_metrics=test_results, train_time=train_time,
    csv_save_path= f"./ModelsEvaluation/7_EfficientNetV2B0/7_TrainingValidationMetrics_{model_name}.csv"
)
display_side_by_side(results_df, super_title="Model Evaluation Results")

In [ ]:
# Plot n right and n wrong predictions
from utilities import plot_predictions
plot_predictions(
    model=model,
    class_names=class_names,
    train_dir=train_dir,
    test_data=test_datagen,
    num_images=10,
    file_path=f"./ModelsEvaluation/7_EfficientNetV2B0/7_TestPredictions_{model_name}.png",
)

---

#### **🟨 Google Collab**

In [ ]:
# Save Google Collab Workspace
# Source: https://stackoverflow.com/questions/48774285/how-to-download-file-created-in-colaboratory-workspace
# !zip -r /content/ModelCallbacks.zip /content/ModelCallbacks

In [ ]:
# !zip -r /content/ModelsEvaluation.zip /content/ModelsEvaluation

In [ ]:
# from google.colab import files
# files.download("/content/ModelCallbacks.zip")

In [ ]:
# files.download("/content/ModelsEvaluation.zip")

---

# **🔗 Bibliography/References**

**[[1]](https://arxiv.org/pdf/1905.11946)** Tan, M., & Le, Q. (2020). EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks. https://arxiv.org/pdf/1905.11946